In [22]:
import os
import pandas as pd
os.getcwd()

'C:\\Users\\regis\\OneDrive\\Dokumente\\GitHub\\wake_effect\\delta_wind_velocity'

In [53]:
# load data - two data frames
wind_dir = pd.read_csv('C:/Users/regis/OneDrive/Dokumente/GitHub/wake_effect/data/Wind_direction.csv')
wind_v = pd.read_csv('C:/Users/regis/OneDrive/Dokumente/GitHub/wake_effect/data/Windfarm.csv')

,Unnamed: 0,10.0,40.0,60.0,80.0,100.0
0,2013-08-01 00:00:00,NaN,NaN,129.7,110.8,113.0
1,2013-08-01 00:10:00,NaN,NaN,111.9,113.0,115.5
2,2013-08-01 00:20:00,NaN,NaN,116.3,110.2,112.2
3,2013-08-01 00:30:00,NaN,NaN,115.2,112.6,115.9
4,2013-08-01 00:40:00,NaN,NaN,118.4,116.6,118.8
...,...,...,...,...,...,...
52555,2014-07-31 23:10:00,NaN,NaN,NaN,99.8,101.8
52556,2014-07-31 23:20:00,NaN,NaN,NaN,103.0,106.0
52557,2014-07-31 23:30:00,NaN,NaN,NaN,104.5,106.9
52558,2014-07-31 23:40:00,NaN,NaN,NaN,106.4,108.5


In [47]:
# merge two dataframes
df = wind_dir.merge(wind_v, left_on='Unnamed: 0', right_on='Unnamed: 0')

,Unnamed: 0,10.0,40.0,60.0,80.0,100.0,WTG1,WTG2,WTG3,WTG4,...,WTG23,WTG24,WTG25,WTG26,WTG27,WTG28,WTG29,WTG30,WTG31,WTG32
0,2013-08-01 00:00:00,NaN,NaN,129.7,110.8,113.0,11.3,12.4,10.4,9.8,...,13.4,12.5,12.2,12.6,12.5,12.8,13.8,12.9,12.4,8.9
1,2013-08-01 00:10:00,NaN,NaN,111.9,113.0,115.5,11.6,11.9,10.4,10.7,...,12.7,13.0,12.5,14.4,12.4,12.8,13.4,13.1,13.2,9.8
2,2013-08-01 00:20:00,NaN,NaN,116.3,110.2,112.2,11.8,12.5,10.8,10.6,...,12.2,12.1,11.7,12.2,12.0,12.2,13.2,12.4,12.3,8.7
3,2013-08-01 00:30:00,NaN,NaN,115.2,112.6,115.9,11.7,11.6,10.6,10.4,...,12.8,13.2,10.9,12.4,12.3,12.3,12.4,11.8,10.5,8.6
4,2013-08-01 00:40:00,NaN,NaN,118.4,116.6,118.8,10.5,11.0,8.6,8.8,...,12.6,12.7,10.9,11.9,12.0,12.5,10.0,12.0,11.5,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,2014-07-31 23:10:00,NaN,NaN,NaN,99.8,101.8,8.5,8.9,7.5,7.4,...,9.3,9.6,8.9,9.9,9.6,9.7,9.3,9.4,9.2,5.1
52556,2014-07-31 23:20:00,NaN,NaN,NaN,103.0,106.0,8.3,9.3,7.6,7.3,...,9.2,9.6,9.5,9.5,9.1,9.4,9.3,9.9,9.6,5.1
52557,2014-07-31 23:30:00,NaN,NaN,NaN,104.5,106.9,9.5,9.4,8.2,7.7,...,9.6,10.1,9.5,10.1,9.7,9.9,9.7,10.0,9.8,5.4
52558,2014-07-31 23:40:00,NaN,NaN,NaN,106.4,108.5,9.5,9.8,8.3,7.6,...,10.0,9.9,9.7,10.0,10.0,10.2,9.8,10.2,9.9,5.3


In [48]:
# Drop columns with inconsistent wind_dir records. Get mean wind direction per angle (wind_dir),
# between average of 80.0 and 100.0. Group wind_dir by steps of 10.
df = df.drop(columns = ['10.0'])
df = df.drop(columns = ['40.0'])
df = df.drop(columns = ['60.0'])
df['wind_dir'] = ((df['80.0'] + df['100.0'])/2).round(-1)

,Unnamed: 0,80.0,100.0,WTG1,WTG2,WTG3,WTG4,WTG5,WTG6,WTG7,...,WTG24,WTG25,WTG26,WTG27,WTG28,WTG29,WTG30,WTG31,WTG32,wind_dir
0,2013-08-01 00:00:00,110.8,113.0,11.3,12.4,10.4,9.8,9.9,10.5,11.4,...,12.5,12.2,12.6,12.5,12.8,13.8,12.9,12.4,8.9,110.0
1,2013-08-01 00:10:00,113.0,115.5,11.6,11.9,10.4,10.7,10.6,10.7,12.0,...,13.0,12.5,14.4,12.4,12.8,13.4,13.1,13.2,9.8,110.0
2,2013-08-01 00:20:00,110.2,112.2,11.8,12.5,10.8,10.6,10.4,10.5,11.9,...,12.1,11.7,12.2,12.0,12.2,13.2,12.4,12.3,8.7,110.0
3,2013-08-01 00:30:00,112.6,115.9,11.7,11.6,10.6,10.4,10.1,10.3,11.7,...,13.2,10.9,12.4,12.3,12.3,12.4,11.8,10.5,8.6,110.0
4,2013-08-01 00:40:00,116.6,118.8,10.5,11.0,8.6,8.8,10.6,10.8,10.5,...,12.7,10.9,11.9,12.0,12.5,10.0,12.0,11.5,8.0,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,2014-07-31 23:10:00,99.8,101.8,8.5,8.9,7.5,7.4,6.6,7.6,8.5,...,9.6,8.9,9.9,9.6,9.7,9.3,9.4,9.2,5.1,100.0
52556,2014-07-31 23:20:00,103.0,106.0,8.3,9.3,7.6,7.3,6.5,7.7,8.2,...,9.6,9.5,9.5,9.1,9.4,9.3,9.9,9.6,5.1,100.0
52557,2014-07-31 23:30:00,104.5,106.9,9.5,9.4,8.2,7.7,7.6,8.5,9.6,...,10.1,9.5,10.1,9.7,9.9,9.7,10.0,9.8,5.4,110.0
52558,2014-07-31 23:40:00,106.4,108.5,9.5,9.8,8.3,7.6,7.7,8.4,9.6,...,9.9,9.7,10.0,10.0,10.2,9.8,10.2,9.9,5.3,110.0


In [51]:
df_ready = df.groupby(['wind_dir']).mean()
df_ready

,80.0,100.0,WTG1,WTG2,WTG3,WTG4,WTG5,WTG6,WTG7,WTG8,...,WTG23,WTG24,WTG25,WTG26,WTG27,WTG28,WTG29,WTG30,WTG31,WTG32
wind_dir,,,,,,,,,,,,,,,,,,,,,
0.0,0.712000,5.827000,3.800000,3.733333,3.833333,3.333333,3.666667,3.300000,3.933333,3.033333,...,4.133333,3.433333,4.266667,3.300000,4.266667,3.433333,4.200000,4.700000,4.466667,4.166667
10.0,9.021962,13.727231,4.342308,4.269231,4.500000,4.146154,4.311538,3.457692,4.300000,3.573077,...,3.530769,3.476923,4.003846,3.411538,4.065385,3.453846,3.807692,4.203846,4.042308,3.730769
20.0,18.866000,23.276667,4.274603,4.246032,4.611111,4.174603,4.344444,3.139683,4.273016,3.517460,...,3.546032,3.842857,4.211111,3.998413,4.155556,4.096825,4.073016,4.420635,4.219048,4.030159
30.0,28.940611,32.519333,5.076111,4.979444,5.407222,4.450000,5.050556,3.923333,5.108889,4.126111,...,4.812222,5.107222,5.063889,5.088333,5.020556,5.023333,4.842222,5.350556,5.067778,4.929444
40.0,39.233612,42.157105,5.742584,5.626555,6.004785,4.956699,5.623684,5.189474,5.781340,5.252392,...,5.675359,5.819617,5.733493,5.756699,5.624641,5.722488,5.607177,6.147129,5.778469,5.586603
50.0,49.302529,51.988967,6.204110,6.080400,6.387777,5.912434,5.440991,6.037092,6.206006,5.763330,...,6.184194,6.306217,6.132350,6.262803,6.137197,6.243414,6.117703,6.618757,6.206639,5.991675
60.0,59.418626,61.981180,6.765207,6.689418,6.117396,6.616445,5.163951,6.335973,6.777198,6.264522,...,6.815226,6.941188,6.709136,6.829577,6.726342,6.866692,6.688390,7.151542,6.808451,6.560754
70.0,69.465417,71.872578,7.554560,7.504409,6.258880,7.389511,6.198133,6.773244,7.530916,6.845316,...,7.686738,7.728960,7.345831,7.568533,7.492480,7.703947,7.447520,7.905671,7.592889,7.290507
80.0,79.009276,81.411873,8.111379,8.128594,7.919780,7.257705,7.790068,7.535824,8.080885,7.442252,...,8.365020,8.330842,7.975786,8.216962,8.162350,8.320551,8.064545,8.472388,8.248522,7.877792
